In [61]:
import numpy as np
from astropy import constants as cst

### Physical constants

In [ ]:
## Physical constants
kB = cst.k_B.cgs.value          ## Boltzmann constant [erg/K]
mH = cst.u.to('g').value        ## mass H atom = atomic mass unit [g]

## Other constants
pi = np.pi


#-- GRAIN PARAMETERS FOR H2 FORMATION AND CR IONISATION
rGr = 1.0E-5        ## grain radius [cm]
nGr = 1.5e-12       ## grain number density/H2 (assuming gas/dust = 200, rho = 3.5 g/cm^3)
w = 0.5             ## grain albedo
AUV_AV = 4.65
stckH = 0.3         ## sticking coefficient for H atoms

In [6]:
print(cst.k_B.cgs.value)

1.380649e-16


### Reading input file

In [ ]:
def readinput(file):
    

    Haccr = stckH *pi*(rGr**2.0)*ρ*nGr*(8.0*kB*T/(pi*mH))**0.5
    return ρ, T, δ, Av, Haccr

In [38]:
'''
TODO 
om te vermijden dat er kei veel moet geloopt worden over elke lijst van elke input parameter, 
kan er gesplit worden gewoon op spatie, en dan een ander keyword meegeven om te beslissen of alle
params gecombineerd worden, of dat het per model is.

'''



infile = '/lhome/silkem/ChemTorch/ChemTorch/input/input.txt'

filelines = list()

with open(infile, 'r') as f:
    lines = f.readlines()
    

print(lines)


for i in range(len(lines)):
    lines[i]=lines[i].replace(' \n', '')
    lines[i]=lines[i].replace('\n', '')
    lines[i]=lines[i].split(' ')

    if lines[i][0] == 'dens':
        dens_idx = i
        dens_line = lines[i][2:]

    if lines[i][0] == 'temp':
        temp_idx = i
        temp_line = lines[i][2:]

    if lines[i][0] == 'delta':
        delta_idx = i
        delta_line = lines[i][2:]

    if lines[i][0] == 'Av':
        Av_idx = i
        Av_line = lines[i][2:]


print(dens_line)
print(temp_line)

if len(dens_line) > 1 and dens_line[1] == ':':
    type = 'single'

if len(dens_line) > 1 and dens_line[1] == ';':
    type = 'combine'

else: type = 'none'

print(type)

input = dict()

['### Inputfile TorchChem ###\n', '### ------------------- ###\n', '\n', '## Use : to split input to create specific models.\n', '## Use ; to cross all input values.\n', '\n', '\n', '## --\n', '\n', '# density [Msol/yr]\n', 'dens = 1e-6 ; 1e-5\n', '\n', '# temperature [K]\n', 'temp = 2000\n', '\n', '# overall dilution of the radiation field\n', 'delta = 1\n', '\n', '# species-specific extinction (connected to optical depth)\n', 'Av = 1\n', '\n', '## --']
['1e-6', ';', '1e-5']
['2000']
combine


### Three body reactions

In [136]:
def read_rate_file():

    loc = 'rates/rate16_IP_2330K_AP_6000K.rates'

    rates = dict()
    with open(loc, 'r') as f:
        lines = f.readlines()
        for i in range(len(lines)):
            line = lines[i].split(':')
            rates[int(line[0])] = line[1:]
    
    type = list()
    α = list()
    β = list()
    γ = list()
    for rate in rates:
        type.append((rates[rate][0]))
        α.append(float(rates[rate][8]))
        β.append(float(rates[rate][9]))
        γ.append(float(rates[rate][10]))

    return rates, type, np.array(α), np.array(β), np.array(γ)

In [104]:
# Read the rates file and return the rate coefficients per reaction: 
#               reaction no.:type:R1:R2:P1:P2:P3:P4:NE:α : β : γ: Tl :Tu:ST:ACC:REF
# indices                 0    1  2  3  4  5  6  7  8  9  10  11  12  13 14 15 

In [137]:
rates, type, α, β, γ = read_rate_file()

In [140]:
no = 1004
α[no-1], type[no-1]

(1.5e-08, 'DR')

In [127]:
rates[1004]

['DR',
 'C2H3+',
 'e-',
 'C2',
 'H',
 'H2',
 '',
 '2',
 '1.50e-08',
 '-0.84',
 '0.0',
 '50',
 '1000',
 'L',
 'A',
 '"10.1051/0004-6361',
 '20020882"',
 '"06_3581D.notes"',
 '2.87e-08',
 '-1.38',
 '0.0',
 '1000',
 '30000',
 'L',
 'A',
 '"10.1051/0004-6361',
 '20020882"',
 '"06_3581D.notes"',
 '\n']

In [116]:
def read_specs_file(chemtype):

    loc = 'rates/rate16_IP_6000K_'+chemtype+'rich_mean_Htot.specs'

    # specs = dict()

    specs = np.loadtxt(loc, skiprows=1,   max_rows=466, usecols=(1), dtype=str)
    consv = np.loadtxt(loc, skiprows=468, max_rows=2  , usecols=(1), dtype=str)
    parnt = np.loadtxt(loc, skiprows=471,               usecols=(0), dtype=str)
    

    return specs, parnt, consv

In [117]:
specs, parnt, consv = read_specs_file('C')

In [119]:
len(consv), specs

(2,
 array(['H', 'H+', 'H-', 'H2+', 'H3+', 'He', 'He+', 'HeH+', 'C-', 'C+',
        'C', 'CH-', 'CH+', 'CH', 'CH2+', 'N+', 'CH2', 'N', 'CH3', 'NH',
        'NH+', 'CH3+', 'O+', 'NH2+', 'CH4+', 'O-', 'NH2', 'O', 'CH4',
        'OH-', 'OH', 'OH+', 'CH5+', 'NH3+', 'NH3', 'NH4+', 'H2O', 'H2O+',
        'F', 'H3O+', 'F+', 'HF', 'HF+', 'H2F+', 'Na', 'Na+', 'C2-', 'C2+',
        'Mg', 'Mg+', 'C2', 'C2H-', 'C2H+', 'C2H', 'CN', 'C2H2', 'C2H2+',
        'CN-', 'CN+', 'HCN', 'C2H3', 'C2H3+', 'HCN+', 'HNC', 'N2', 'C2H4+',
        'Si+', 'H2CN', 'H2NC+', 'N2+', 'Si', 'CO', 'CO+', 'C2H4', 'HCNH+',
        'C2H5+', 'HCO', 'SiH+', 'HCO+', 'CH2NH', 'SiH', 'C2H5', 'HOC+',
        'N2H+', 'CH2NH2+', 'SiH2', 'CH3CH3+', 'NO', 'H2CO', 'CH4N+',
        'H2CO+', 'CH3CH3', 'SiH2+', 'NO+', 'P+', 'P', 'H3CO+', 'HNO+',
        'SiH3+', 'HNO', 'CF+', 'C2H7+', 'SiH3', 'O2', 'SiH4', 'PH', 'S',
        'H2NO+', 'CH3OH', 'SiH4+', 'O2+', 'S-', 'O2-', 'PH+', 'CH3OH+',
        'S+', 'CH3OH2+', 'O2H', 'PH2+', 'HS', 'HS+',